In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import datetime
from time import sleep
import geopandas as gpd

In [2]:
def queryBicipublica2(biciID):
    try:
        url = 'http://epok.buenosaires.gob.ar/getObjectContent/?id=estaciones_de_bicicletas%7C' + biciID
        response = urllib.urlopen(url)
        rawData = response.read().decode("utf-8")
        parsedData = json.loads(rawData)

        bicisDisponibles = int(parsedData['contenido'][3]['valor'])
        posicionesDisponibles = int(parsedData['contenido'][4]['valor'])
        totales = int(parsedData['contenido'][5]['valor'])
    
    except:
        bicisDisponibles = np.nan
        posicionesDisponibles = np.nan
        totales = np.nan
        
    
    return {'bicisDisponibles':bicisDisponibles,'posicionesDisponibles':posicionesDisponibles,'totales':totales}

In [3]:
#base madre
geometria = pd.read_csv('../data/estaciones-de-bicicletas-publicas.csv',sep=';')
#convertir en texto los id de las estaciones para el query
geometria.NRO_EST = ['0' + str(i) if len(str(i)) == 1 else str(i) for i in geometria.NRO_EST]
#remover columnas innecesarias
geometria = geometria.loc[:,['X','Y','NOMBRE','NRO_EST']]
geometria.head()



,X,Y,NOMBRE,NRO_EST
0,-58.392452,-34.583133,FACULTAD DE DERECHO,01
1,-58.374822,-34.592589,RETIRO,02
2,-58.368918,-34.611242,ADUANA,03
3,-58.368950,-34.601721,PLAZA ROMA,04
4,-58.369360,-34.627947,PARQUE LEZAMA,06


In [4]:
#modulo tiempo NUEVOOOO
hora = datetime.datetime.now().hour
#poner un dia limite
diaLimite = 21

fechas = []
bicisDisponibles = []
posicionesDisponibles = []
totales = []
biciIDs = []



#inicia el loop para cargar data
#cambiar hora 10a 0 y hour por day
while True:
    if datetime.datetime.now().hour == 22:
        print 'hora inicio'
        #mientras el dia actual sea menor al limite
        while datetime.datetime.now().day < diaLimite:
            
            fecha = datetime.datetime.now()
            print 'fecha:',fecha
            #inicia el proceso para cada numero de estacion
            for biciID in geometria.NRO_EST:
                #agrego la id de la bici a la lista
                biciIDs.append(biciID)
                #creo el diccionario con datos
                diccio = queryBicipublica2(biciID)
                #extraigo disponibles y total
                bicisDisponibles.append(diccio['bicisDisponibles'])
                posicionesDisponibles.append(diccio['posicionesDisponibles'])
                totales.append(diccio['totales'])
                
                #agrego fecha
                fechas.append(fecha)
                
            #espera 15 min
            print fecha
            sleep(60*15)
        break
    else:
        sleep(60)

datos = pd.DataFrame({'NRO_EST':biciIDs,
                      'fecha':fechas,
                      'bicisDisponibles':bicisDisponibles,
                      'posicionesDisponibles':posicionesDisponibles,
                      'totales':totales
                      })

hora inicio
fecha: 2017-09-19 22:51:55.460997
2017-09-19 22:51:55.460997
fecha: 2017-09-19 23:15:29.162675
2017-09-19 23:15:29.162675
fecha: 2017-09-19 23:34:21.675951
2017-09-19 23:34:21.675951
fecha: 2017-09-19 23:51:36.619720
2017-09-19 23:51:36.619720
fecha: 2017-09-20 00:07:54.142386
2017-09-20 00:07:54.142386
fecha: 2017-09-20 00:23:51.298097
2017-09-20 00:23:51.298097
fecha: 2017-09-20 00:39:39.652832
2017-09-20 00:39:39.652832
fecha: 2017-09-20 00:55:29.199217
2017-09-20 00:55:29.199217
fecha: 2017-09-20 01:11:16.693750
2017-09-20 01:11:16.693750
fecha: 2017-09-20 01:27:04.710801
2017-09-20 01:27:04.710801
fecha: 2017-09-20 01:43:02.050960
2017-09-20 01:43:02.050960
fecha: 2017-09-20 01:58:49.839608
2017-09-20 01:58:49.839608
fecha: 2017-09-20 02:14:37.671382
2017-09-20 02:14:37.671382
fecha: 2017-09-20 02:30:27.133038
2017-09-20 02:30:27.133038
fecha: 2017-09-20 02:46:13.957412
2017-09-20 02:46:13.957412
fecha: 2017-09-20 03:01:58.005312
2017-09-20 03:01:58.005312
fecha: 2017-

In [5]:
print 'datos', datos.shape
print 'geometria',geometria.shape

datos (17836, 5)
geometria (196, 4)


In [6]:
data = geometria.merge(datos)
print data.shape
data.head()

(17836, 8)


,X,Y,NOMBRE,NRO_EST,bicisDisponibles,fecha,posicionesDisponibles,totales
0,-58.392452,-34.583133,FACULTAD DE DERECHO,01,3.0,2017-09-19 22:51:55.460997,14.0,16.0
1,-58.392452,-34.583133,FACULTAD DE DERECHO,01,3.0,2017-09-19 23:15:29.162675,14.0,16.0
2,-58.392452,-34.583133,FACULTAD DE DERECHO,01,3.0,2017-09-19 23:34:21.675951,14.0,16.0
3,-58.392452,-34.583133,FACULTAD DE DERECHO,01,3.0,2017-09-19 23:51:36.619720,14.0,16.0
4,-58.392452,-34.583133,FACULTAD DE DERECHO,01,2.0,2017-09-20 00:07:54.142386,16.0,16.0


In [8]:
data.to_csv('../data/data18sept.csv')

In [10]:
barrios = gpd.read_file('../data/barrios-rar/barrios_badata.shp')
barrios = barrios.to_crs(epsg=4326)
barrios.head()

,AREA,BARRIO,COMUNA,PERIMETRO,geometry
0,3.118101e+06,CHACARITA,15.0,7725.695228,"POLYGON ((-58.452220824502 -34.59643594220518,..."
1,2.229829e+06,PATERNAL,15.0,7087.513295,POLYGON ((-58.46497711478177 -34.5970050426002...
2,3.613584e+06,VILLA CRESPO,15.0,8132.699348,POLYGON ((-58.4231548125714 -34.59827443880985...
3,3.399596e+06,VILLA DEL PARQUE,11.0,7705.389797,POLYGON ((-58.49400885920002 -34.6153122984289...
4,4.050752e+06,ALMAGRO,5.0,8537.901368,POLYGON ((-58.41227214621448 -34.6145629260479...


In [11]:
biciShape = gpd.read_file('../data/estaciones-de-bicicletas-rar/estaciones_de_bicicletas.shp')
biciShape = biciShape.to_crs(epsg=4326)
print biciShape.shape
biciShape.head()

(196, 9)


,AUTOMAT,DIRE_NORM,DOMICILIO,Horario,IMAGEN,NOMBRE,NRO_EST,OBSERV,geometry
0,AUTOMÁTICA,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Facultad de Derecho,1,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.39185516097447 -34.58358035902182)
1,AUTOMÁTICA,"DEL LIBERTADOR AV. y RAMOS MEJIA, JOSE MARIA, ...",Av. Dr.Jose Ramos Mejia y Del Libertador Av,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Retiro,2,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.37422545676812 -34.59303579060376)
2,AUTOMÁTICA,"HUERGO, ING. AV. y BELGRANO AV.",Av. Ing.Huergo y Av. Belgrano,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Aduana,3,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.36832159551064 -34.61168855013559)
3,AUTOMÁTICA,LAVALLE y BOUCHARD,Lavalle y Bouchard,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Plaza Roma,4,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.36835420317743 -34.60216783080737)
4,AUTOMÁTICA,"GARCIA, MARTIN AV. e IRALA",Av Martín García e Irala,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Parque Lezama,6,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.36876396832855 -34.62839319029082)


In [12]:
bicisConBarrio = gpd.sjoin(biciShape, barrios, how="inner", op='intersects')
print bicisConBarrio.shape
print bicisConBarrio.BARRIO.isnull().sum()

(196, 14)
0


In [13]:
bicisConBarrio.NRO_EST = ['0' + str(i) if len(str(i)) == 1 else str(i) for i in biciShape.NRO_EST]
bicisConBarrio = bicisConBarrio.merge(datos)
bicisConBarrio.shape

In [15]:
bicisConBarrio.fecha = bicisConBarrio.fecha.map(str)

In [16]:
bicisConBarrio.to_file('../data/18septiembre')

In [17]:
bicisConBarrio.head()

,AUTOMAT,DIRE_NORM,DOMICILIO,Horario,IMAGEN,NOMBRE,NRO_EST,OBSERV,geometry,index_right,AREA,BARRIO,COMUNA,PERIMETRO,bicisDisponibles,fecha,posicionesDisponibles,totales
0,AUTOMÁTICA,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Facultad de Derecho,01,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.39185516097447 -34.58358035902182),44,6.140873e+06,RECOLETA,2.0,21246.611702,3.0,2017-09-19 22:51:55.460997,14.0,16.0
1,AUTOMÁTICA,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Facultad de Derecho,01,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.39185516097447 -34.58358035902182),44,6.140873e+06,RECOLETA,2.0,21246.611702,3.0,2017-09-19 23:15:29.162675,14.0,16.0
2,AUTOMÁTICA,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Facultad de Derecho,01,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.39185516097447 -34.58358035902182),44,6.140873e+06,RECOLETA,2.0,21246.611702,3.0,2017-09-19 23:34:21.675951,14.0,16.0
3,AUTOMÁTICA,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Facultad de Derecho,01,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.39185516097447 -34.58358035902182),44,6.140873e+06,RECOLETA,2.0,21246.611702,3.0,2017-09-19 23:51:36.619720,14.0,16.0
4,AUTOMÁTICA,"FIGUEROA ALCORTA, PRES. AV. y BIBILONI, JUAN A.",Av. Pres.Figueroa Alcorta y Juan A.Bibiloni,Estación automática: disponibilidad las 24 horas,estacion_automatica.png,Facultad de Derecho,01,Abril 2015 (pasó de ser Manual a Automática),POINT (-58.39185516097447 -34.58358035902182),44,6.140873e+06,RECOLETA,2.0,21246.611702,2.0,2017-09-20 00:07:54.142386,16.0,16.0
